# Initialization 
This notebook aims to help gain a better understanding of indicator functions by defining a neural network and training the neural network using a backward pass, which involves using an indicator function.

### Imports 
Import the libraries needed to define the indicator function, neural network, and backward pass

In [1]:
import numpy as np
import matplotlib.pyplot as plt

### Define Parameter Function
Define a function that initializes the value of the weights and biases

In [2]:
def init_params(K, D, sigma_sq_omega):
  # Set seed in order to consistently obtain the same random number
  np.random.seed(0)

  # Input layer
  D_i = 1
  # Output layer
  D_o = 1

  # Construct an empty list for the weights and biases
  all_weights = [None] * (K+1)
  all_biases = [None] * (K+1)

  # Construct an array structure and intialize the weights and biases of the input and output layers. Add varaince to the weights
  all_weights[0] = np.random.normal(size=(D, D_i))*np.sqrt(sigma_sq_omega)
  all_weights[-1] = np.random.normal(size=(D_o, D)) * np.sqrt(sigma_sq_omega)
  all_biases[0] = np.zeros((D,1))
  all_biases[-1]= np.zeros((D_o,1))

  # Construct intermediate hidden layers
  for layer in range(1,K):
    all_weights[layer] = np.random.normal(size=(D,D))*np.sqrt(sigma_sq_omega)
    all_biases[layer] = np.zeros((D,1))

  return all_weights, all_biases

### Define the Rectified Linear Unit (ReLU) Function

In [3]:
def ReLU(preactivation):
  activation = preactivation.clip(0.0)
  return activation

### Define Neural Network
Define a function that computes a neural network given the value of the input, weights, and biases.

In [4]:
def compute_network_output(net_input, all_weights, all_biases):

  # Retrieve number of layers
  K = len(all_weights)-1

  # Store the pre-activations (all_f) and the activations (all_h)
  all_f = [None] * (K+1)
  all_h = [None] * (K+1)

  all_h[0] = net_input

  # Calculate the pre-activation and activation for each hidden layer
  for layer in range(K):
      # Compute the pre-activation function
      all_f[layer] = all_biases[layer] + np.matmul(all_weights[layer], all_h[layer])
      # Compute the activation function
      all_h[layer+1] = ReLU(all_f[layer])

  # Compute the output of the neural network from the last hidden layer
  all_f[K] = all_biases[K] + np.matmul(all_weights[K], all_h[K])

  # Retrieve the output
  net_output = all_f[K]

  return net_output, all_f, all_h

### Define Hyperparameters 
Define the hyperparameters (number of layers, number of neurons per layer, number of input layers, number of output layers) for the neural network model

In [5]:
# Define the number of layers
K = 5
# Define the number of neurons per layer
D = 8
# Define the number of input layers
D_i = 1
# Define the number of output layer
D_o = 1

### Define Variance 
Define the variance of the initial weight values

In [6]:
sigma_sq_omega = 1.0

### Initialize Neural Network Parameters

In [7]:
all_weights, all_biases = init_params(K,D,sigma_sq_omega)

### Compute Neural Network

In [8]:
n_data = 1000
data_in = np.random.normal(size=(1,n_data))
net_output, all_f, all_h = compute_network_output(data_in, all_weights, all_biases)

### Compute the Standard Deviation of the Hidden Units in each Hidden Layer

In [9]:
for layer in range(1,K+1):
  print("Layer %d, std of hidden units = %3.3f"%(layer, np.std(all_h[layer])))

Layer 1, std of hidden units = 0.811
Layer 2, std of hidden units = 1.472
Layer 3, std of hidden units = 4.547
Layer 4, std of hidden units = 8.896
Layer 5, std of hidden units = 10.106


### Define the Least Squares Loss Function

In [10]:
def least_squares_loss(net_output, y):
  return np.sum((net_output-y) * (net_output-y))

### Define Derivative of Loss with Respect to Output
Define a function that computes the derivative of the loss function with respect to the output of the neural network for the backward pass

In [11]:
def d_loss_d_output(net_output, y):
    return 2*(net_output -y)

### Define an Indicator Function
Define an indicator function for the backward pass that returns a 1 if the input is greater than equal to 0 and a 0 if the input is less than 0

In [12]:
def indicator_function(x):
  x_in = np.array(x)
  x_in[x_in>=0] = 1
  x_in[x_in<0] = 0
  return x_in

### Define Backward Pass
Define a function to compute the main backward pass

In [13]:
def backward_pass(all_weights, all_biases, all_f, all_h, y):
  # Retrieve number of layers
  K = len(all_weights) - 1

  # Store the derivatives dl_dweights and dl_dbiases in lists
  all_dl_dweights = [None] * (K+1)
  all_dl_dbiases = [None] * (K+1)
  # Store the derivatives of the loss with respect to the activation and preactivations in lists
  all_dl_df = [None] * (K+1)
  all_dl_dh = [None] * (K+1)

  # Compute derivatives of the loss with respect to the network output
  all_dl_df[K] = np.array(d_loss_d_output(all_f[K],y))

  # Compute the backward pass
  for layer in range(K,-1,-1):
    # Calculate the derivatives of the loss with respect to the biases at layer
    all_dl_dbiases[layer] = np.array(all_dl_df[layer])
    # Calculate the derivatives of the loss with respect to the weights at layer
    all_dl_dweights[layer] = np.matmul(all_dl_df[layer], all_h[layer].transpose())

    # Calculate the derivatives of the loss with respect to the activations
    all_dl_dh[layer] = np.matmul(all_weights[layer].transpose(), all_dl_df[layer])
   
    if layer > 0:
       # Calculate the derivatives of the loss with respect to the pre-activation f
      all_dl_df[layer-1] = indicator_function(all_f[layer-1]) * all_dl_dh[layer]

  return all_dl_dweights, all_dl_dbiases, all_dl_dh, all_dl_df

### Define Hyperparameters 
Define the hyperparameters (number of layers, number of neurons per layer, number of input layers, number of output layers) for the neural network model

In [14]:
# Define the number of layers
K = 5
# Define the number of neurons per layer
D = 8
# Define input layer
D_i = 1
# Define output layer
D_o = 1

### Define Variance 
Define the variance of the initial weight values

In [15]:
sigma_sq_omega = 1.0

### Initialize Neural Network Parameters

In [16]:
all_weights, all_biases = init_params(K,D,sigma_sq_omega)

### Initialize Gradient Array Structure
Initialize an array structure, which stores the computed gradients of the backward pass

In [17]:
n_data = 100
aggregate_dl_df = [None] * (K+1)
for layer in range(1,K):
  # Store the gradients for every data point in the 3D array
  aggregate_dl_df[layer] = np.zeros((D,n_data))

### Compute Gradients 
Compute the derivatives of the parameters for each data point separately (gradient) and store it in the intiaialized gradient array structure

In [18]:
for c_data in range(n_data):
  data_in = np.random.normal(size=(1,1))
  y = np.zeros((1,1))
  net_output, all_f, all_h = compute_network_output(data_in, all_weights, all_biases)
  all_dl_dweights, all_dl_dbiases, all_dl_dh, all_dl_df = backward_pass(all_weights, all_biases, all_f, all_h, y)
  for layer in range(1,K):
    aggregate_dl_df[layer][:,c_data] = np.squeeze(all_dl_df[layer])

### Compute the Standard Deviation of the Gradient for each Hidden Layer

In [19]:
for layer in reversed(range(1,K)):
  print("Layer %d, std of dl_dh = %3.3f"%(layer, np.std(aggregate_dl_df[layer].ravel())))

Layer 4, std of dl_dh = 56.472
Layer 3, std of dl_dh = 109.132
Layer 2, std of dl_dh = 340.657
Layer 1, std of dl_dh = 446.654
